# Train a Recommendation Model using Pytorch and Amazon SageMaker
AI-driven personalization in media has become essential for delivering tailored content experiences that keep audiences engaged in today's media streaming experience. By analyzing insights about user data including viewing habits, preferences and patterns, AI algorithms can create highly accurate content recommendations and customize media experiences in real-time. Additionally, since user preferences and tastes for the media could change over time, the recommendation AI model can be updated to ensure that recommendations remain relevant and valuable, while also helping content creators and publishers better understand their audiences and make data-driven decisions about future content development and distribution strategies.

Today's recommendation systems typically fall into three main categories. First, content-based filtering analyzes the characteristics and metadata of items you've previously enjoyed or engaged with to suggest similar content. Second, collaborative filtering makes predictions about what you might like based on the preferences and actions of users who exhibit similar behavior patterns - this data comes either from explicit ratings or, more commonly, from tracking how users interact with content. Finally, hybrid systems take the best of both approaches, combining content metadata with user interaction data to create more sophisticated recommendations.

In this lab, we will walk you through how you could build a hybrid recommendation system using a popular recommendation model algorithm called Two Tower Model. 

## Two Tower Model
The basic principle for two-tower model architecture is about training 2 embedding models using two separate neural networks called towers. In the context of media recommendation, you consider these two towers as separate models one representing the users, known as the query tower, and one representing the shows/media, known as the candidate tower. During training, each tower learns to transform an arbitrary set of input features into vectors known as embeddings. The dimension of these embeddings must be the same for both the users and the items as finally the similarity between them is measured using the dot product. 

This hybrid approach has been studied and proven effective across recommendation and personalization use cases, especially those that involves very large number of user iterations and item catalogs. For more details about two tower model literatures, please refer to these papers [here](https://dl.acm.org/doi/abs/10.1145/3366423.3380130) and [here](https://dlp-kdd.github.io/assets/pdf/DLP-KDD_2021_paper_4.pdf) and [here](https://arxiv.org/html/2403.00802v1). 

Here's a simplified diagram that depicts the architecture:

![two-tower-architecture](./img/two-tower-architecture.png)

## Workshop
The goal of this lab is to train a two tower model. As depicted in the architecture diagram above, there are 2 embeddings tables created as part of this particular training job. 

In the next lab, we'll show how you could use these embeddings in action. Specifically, we'll be using them to perform similarity search to find relevant shows for the user based on their viewing history. 


## ML Framework
[TorchRec](https://pytorch.org/tutorials/intermediate/torchrec_intro_tutorial.html) TorchRec is a specialized PyTorch domain library designed for building large-scale recommendation systems. It powers many production recommendation models at Meta, including models with up to 3 trillion parameters.
TorchRec provides optimization including efficient handling of sparse features such as **[JaggedTensor](https://pytorch.org/torchrec/datatypes-api-reference.html#torchrec.sparse.jagged_tensor.JaggedTensor)** and **[KeyedJaggedTensor](https://pytorch.org/torchrec/datatypes-api-reference.html#torchrec.sparse.jagged_tensor.KeyedJaggedTensor)**, pre-built modules for common recommendation architectures. In addition, TorchRec provides an advanced scaling capabilities that enables efficient and scalable distribution of large embedding tables across multiple GPUs and nodes. Watch this [youtube video](https://www.youtube.com/watch?v=cjgj41dvSeQ&t=5s) to learn more about this library

In this lab, we will use this TorchRec library to build a Two Tower model. 

## Dataset
MovieLens is a movie recommendation service that has generated several important datasets used for research in recommender systems and machine learning. The datasets contain user ratings of movies, where users provide ratings on a 5-star scale, along with additional movie and user information.

There are a few variants of the dataset:

**Main Variants:**

* 25M Dataset: Contains 25,000,095 ratings across 62,423 movies from 162,541 users1
* Latest-Full: Includes approximately 33,000,000 ratings and 2,000,000 tag applications for 86,000 movies by 330,975 users6
* 100K Dataset: The oldest version, containing 100,000 ratings from 943 users on 1,682 movies1
* 1M Dataset: Features 1,000,209 ratings with demographic data

We will be using the [movielens](https://grouplens.org/datasets/movielens/) 1M dataset to train the model.



Let's get started!

In [ ]:
%store -r

Import python dependencies

In [ ]:
import sagemaker
import boto3
import time
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorch
from sagemaker.local import LocalSession
from sagemaker import Session
from sagemaker import get_execution_role

Setting up a proper IAM role, a sagemaker session to create a training job.

In [ ]:
role = get_execution_role()
# session = LocalSession(). # Uncomment if you want to test locally. (https://github.com/aws-samples/amazon-sagemaker-local-mode)
# session.config = {'local': {'local_code': True}} # uncomment if you want to test locally 
session = sagemaker.Session()
bucket = session.default_bucket()
sm_training_s3_output_location = f"s3://{bucket}/models/two-tower"
region = session.boto_region_name

In the following step, we'll download and unzip the movielens dataset and use it as the training dataset for the two tower model.

In [ ]:
!wget --no-check-certificate -P datasets https://files.grouplens.org/datasets/movielens/ml-1m.zip && cd datasets && unzip ml-1m.zip

## ML Tranining Data for SageMaker Training job
When creating a training job, you specify the location of training datasets in a data storage of your choice and the data input mode for the job. Amazon SageMaker AI supports Amazon Simple Storage Service (Amazon S3), Amazon Elastic File System (Amazon EFS), and Amazon FSx for Lustre. You can choose one of the input modes to stream the dataset in real time or download the whole dataset at the start of the training job.

Here's an image showing all the input modes and access patterns that SageMaker Training job suports:

![sm training data access](img/sagemaker-training-input-mode.png)

<div class="alert alert-info"> 💡 When submitting a SageMaker training job, it's important to verify the IAM role with the permission to access the dataset that you've decided to use. 
</div>

After downloading the file, we'll upload the training dataset to S3.

In [ ]:
from sagemaker.s3 import S3Uploader

prefix = "data/two-tower"
training_data_s3_uri = S3Uploader.upload(
    local_path="datasets/ml-1m/ratings.dat",
    desired_s3_uri=f"s3://{bucket}/{prefix}",
)

## HyperParameters
Hyperparameters are configuration variables that control the machine learning model's learning process and must be set before training begins. Unlike model parameters that are learned during training, hyperparameters remain fixed throughout the training process. 

Here's a subset of hyperparameters supported for this model. We will configure these parameters to send to the SageMaker training job.



## MLFlow Integration
Machine learning is an iterative process that requires experimenting with various combinations of data, algorithms, and parameters, while observing their impact on model accuracy. 
The complexity of managing and comparing iterative training runs increases with very large models such as foundation models, where experimentation involves not only fine-tuning models but also exploring creative and diverse outputs. 

For tracking all model experimentation, we'll MLflow. Make sure you've completed the [00-start-here.ipynb](00-start-here.ipynb) notebook before submitting a training job. 
Please refer to the notebook to obtain the mlflow ARN for the following step. 

In [ ]:
hyperparams = {
            'epochs' :  50, 
            'batch-size' : 32,
            'num_embeddings' : 1024*1024,
            'embedding_dim' : 64,
            'learning_rate' : 0.01,
            'mlflow-tracking-server-arn' : mlflow_arn,
            }


# AWS Deep Learning Containers
[AWS Deep Learning Containers](https://github.com/aws/deep-learning-containers) (DLCs) are a set of Docker images for training and serving models in TensorFlow, TensorFlow 2, PyTorch and others. Deep Learning Containers provide optimized environments with TensorFlow, Nvidia CUDA (for GPU instances), and Intel MKL (for CPU instances) libraries and are available in the Amazon Elastic Container Registry (Amazon ECR).

Pytorch is one of the most popular AI/ML framework today. If you are using Pytorch framework to train or serve your ML workload, AWS Deep Container got you covered. Rather than building a custom image, you can simply use one of the many SageMaker pre-built containers available [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#sagemaker-framework-containers-sm-support-only)

To retrieve a specific container image, you can directly reference the ECR URI in the github link, or use SageMaker SDK to return the proper URI based on the frameowork version. 


In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework='pytorch',
    region=region,
    version='2.5.1',
    py_version='py311',
    image_scope="training",
    instance_type='ml.p3.2xlarge'
)

# Submitting a Distributed PyTorch SageMaker Training Job
SageMaker Training supports distributed training on a single instance as well as multiple instances, so you can run any size of training at scale. We recommend you to use the framework estimator classes such as PyTorch in the SageMaker Python SDK, which are the training job launchers with various distributed training options. When you create an estimator object, the object sets up distributed training infrastructure, runs the CreateTrainingJob API in the backend, finds the Region where your current session is running, and pulls one of the pre-built AWS deep learning container prepackaged with a number of libraries including deep learning frameworks, distributed training frameworks, and the EFA driver. For more information please refer to [this](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training-get-started.html) documentation.

The following diagram depicts how SageMaker handles a training job:

![sm training arch](img/sagemaker-training-arch-diagram.png)


While this training script is capable of running the training across multiple GPUs, for our lab, we'll use a single GPU instance (p3.2xlarge) to demonstrate the ability. You can use multiple GPU instance (or multiple GPU cores) in your own AWS environment by changing the `instance_count` attribute in the Pytorch estimator class, as shown in the following cell.


In [ ]:
pytorch_estimator = PyTorch(entry_point='train.py',
                            instance_type='ml.p3.2xlarge',
                            # instance_type='local_gpu', # uncomment this to train in the local mode.
                            instance_count=1,
                            image_uri=image_uri,
                            hyperparameters=hyperparams,
                            source_dir='src',
                            role=role, # uses the default SageMaker execution role here, if you have a custom IAM role you should provide it here.
                            output_path=sm_training_s3_output_location,
                            sagemaker_session=session,
                            distribution={
                                "torch_distributed": {
                                    "enabled": True
                                }
                            }) # configuring pytorch distributed training (https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training-get-started.html)


Submit the SM training job asynchronously. This job will take about 10-15 minutes to complete. If you want to wait for the job to complete in the cell, pass `wait`=False to the fit() call below.

In [ ]:
pytorch_estimator.fit( {'train': training_data_s3_uri}, wait=False)

While waiting for the job to complete, let's take a moment to explore the training script [train.py](src/train.py)
*Note:* Most of the training script and the utility functions are adopted from TorchRec's example [here](https://github.com/pytorch/torchrec/blob/main/examples/retrieval/two_tower_train.py).

In [ ]:
%load src/train.py

Wait for the job to complete

In [ ]:
def wait_for_training_job(job_name, sagemaker_client, poll_interval=120):
    """
    Wait for a SageMaker training job to complete.
    
    Args:
        job_name (str): Name of the training job
        sagemaker_client: Boto3 SageMaker client
        poll_interval (int): Time in seconds between status checks
    
    Returns:
        dict: Final job description
    """
    try:
        while True:
            response = sagemaker_client.describe_training_job(
                TrainingJobName=job_name
            )
            status = response['TrainingJobStatus']
            
            if status in ['Completed', 'Failed', 'Stopped']:
                return response
                
            time.sleep(poll_interval)
            
    except Exception as e:
        raise Exception(f"Error waiting for training job: {str(e)}")


In [ ]:
sm_job_name = pytorch_estimator.latest_training_job.job_name

In [ ]:
wait_for_training_job(sm_job_name, session.sagemaker_client, poll_interval=120)

## Track Experiment with SageMaker MLFlow
SageMaker MLflow capability allows users to track experiments across multiple environments including local IDEs, SageMaker Studio, training jobs, and processing jobs and more.
You can also compare model performance, parameters, and metrics across experiments through the MLflow UI which is fully managed in SageMaker Studio. 
The MLFlow integration is configured in the training script `train.py` shown above. Specifically, we use MLFlow API to log training metrics, parameters and artifacts, all grouped in an experiment. Here are the screenshots for the model parameters and training metrics captured in the MLFlow tracking server that we provisioned in this workshop. 

Here's a screenshot on how to navigate to the MLFlow tracking server from SageMaker Studio.

![mlflow ui](img/mlflow-open.png)

To navigate to the experiment for our training job, click on `two-tower-training` experiment as shown in the following:abs

![mlflow exp](img/mlflow-two-tower-exp.png)

Click on the run name on the right hand pane to see the overview:

![mlflow run overview](img/two-tower-training-run.png)

Next, click on `Model metrics` tab to see a visualization of the training loss from the training run:

![training loss metrics](img/two-tower-training-run-loss-metrics.png)

Save the job name and model S3 URI for the next lab.

In [ ]:
%store sm_job_name

In [ ]:
sm_model_s3_url = pytorch_estimator.model_data

In [ ]:
%store sm_model_s3_url

# Next Step
Congratulations! You have completed the training of a two tower model using Pytorch and Distributed training in SageMaker! In the next lab, we'll focus on deploying this trained model using SageMaker inference. Go ahead and navigate to [02-sm-inference.ipynb](02-sm-inference.ipynb) and continue the lab there. 